In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
driver = webdriver.Chrome('/Users/francopiccolo/Utils/chromedriver')

<ipython-input-63-5a3ae8242bdf>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/francopiccolo/Utils/chromedriver')


In [66]:
# Total data 130k
url = 'https://resultados.gob.ar/elecciones/3/{}/1/-1/-1'
section_ids = range(0, 561)

def write_source_to_file(source, filename):
    with open('data/' + str(filename) + '.html', 'w') as f:
        f.write(source)

def wait_for_page_to_load(driver):
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'main'))
    )

for i in section_ids:
    print('Processing: ', i)
    driver.get(url.format(i))
    wait_for_page_to_load(driver)
    write_source_to_file(driver.page_source, i)

Processing:  0
Processing:  1
Processing:  2
Processing:  3
Processing:  4
Processing:  5
Processing:  6
Processing:  7
Processing:  8
Processing:  9
Processing:  10
Processing:  11
Processing:  12
Processing:  13
Processing:  14
Processing:  15
Processing:  16
Processing:  17
Processing:  18
Processing:  19
Processing:  20
Processing:  21
Processing:  22
Processing:  23
Processing:  24
Processing:  25
Processing:  26
Processing:  27
Processing:  28
Processing:  29
Processing:  30
Processing:  31
Processing:  32
Processing:  33
Processing:  34
Processing:  35
Processing:  36
Processing:  37
Processing:  38
Processing:  39
Processing:  40
Processing:  41
Processing:  42
Processing:  43
Processing:  44
Processing:  45
Processing:  46
Processing:  47
Processing:  48
Processing:  49
Processing:  50
Processing:  51
Processing:  52
Processing:  53
Processing:  54
Processing:  55
Processing:  56
Processing:  57
Processing:  58
Processing:  59
Processing:  60
Processing:  61
Processing:  62
Pr

In [36]:
def get_soup_from_file(file):
    with open(file_, 'r') as f:
        file_text = f.read()
    return BeautifulSoup(file_text, 'html.parser')

def parse_soup(soup):
    geographies = soup.find_all('a', {'class': 'RoutedBreadcrumbs__SLink-ccRASu'})
    parties = soup.find_all('div', {'class': 'titleNombreTerr'})
    percentages = soup.find_all('div', {'class': 'porcAgr'})
    votes = soup.find_all('div', {'class': 'numAgr'})

    geographies = [geography.text for geography in geographies]
    parties = [party.text for party in parties]
    percentages = [float(percentage.text.replace(',', '.').replace('%', ''))/100 for percentage in percentages]
    votes = [int(vote.text.replace('.', '')) for vote in votes]
    return geographies, parties, percentages, votes
    
def make_dict(geographies, parties, percentages, votes):
    for party, percentage, votes in zip(parties, percentages, votes):
        if geographies[1] not in data:
            data[geographies[1]] = {'total': {}}
        
        if len(geographies) == 2:
            if party not in data[geographies[1]]['total']:
                data[geographies[1]]['total'][party] = {}
            data[geographies[1]]['total'][party]['percentage'] = percentage
            data[geographies[1]]['total'][party]['votes'] = votes
        else:
            if geographies[2] not in data[geographies[1]]:
                data[geographies[1]][geographies[2]] = {}
            
            if party not in data[geographies[1]][geographies[2]]:
                data[geographies[1]][geographies[2]][party] = {}
            
            data[geographies[1]][geographies[2]][party]['percentage'] = percentage
            data[geographies[1]][geographies[2]][party]['votes'] = votes
        
def make_df(geographies, parties, percentages, votes):

    geography_l2 = 'Total' if len(geographies) < 2 else geographies[1]
    geography_l3 = 'Total' if len(geographies) < 3 else geographies[2]

    return pd.DataFrame({'geography_l1': geographies[0], 'geography_l2': geography_l2,
                    'geography_l3': geography_l3, 'party': parties, 
                    'votes': votes, 'percentage': percentages})

def process_file(file_):
    soup = get_soup_from_file(file_)
    geographies, parties, percentages, votes = parse_soup(soup)
    df = make_df(geographies, parties, percentages, votes)
    return df

In [37]:
# Read files
df = pd.DataFrame(columns=['geography_l1', 'geography_l2', 'geography_l3', 'party', 
                          'votes', 'percentage'])
for i in range(0, 561):
    file_ = 'data/{}.html'.format(i)    
    df = df.append(process_file(file_))

In [38]:
states_mask = df['geography_l3'] == 'Total'

In [42]:
df[states_mask].groupby(['geography_l1', 'geography_l2', 'geography_l3', 'party'])['percentage'].max().reset_index().sort_values(by='percentage', ascending=False)

,geography_l1,geography_l2,geography_l3,party,percentage
115,Argentina,Santiago del Estero,Total,FRENTE CIVICO POR SANTIAGO,0.6350
27,Argentina,Corrientes,Total,ECO + VAMOS CORRIENTES,0.5891
44,Argentina,Formosa,Total,FRENTE DE TODOS,0.5764
56,Argentina,La Rioja,Total,FRENTE DE TODOS,0.5606
40,Argentina,Entre Ríos,Total,JUNTOS POR ENTRE RÍOS,0.5452
...,...,...,...,...,...
98,Argentina,San Luis,Total,TODOS UNIDOS,0.0132
120,Argentina,Santiago del Estero,Total,POR UN SANTIAGO OBRERO,0.0118
36,Argentina,Córdoba,Total,UNIÓN POPULAR FEDERAL,0.0083
46,Argentina,Formosa,Total,PRINCIPIOS Y CONVICCIÓN,0.0078
